## Libreries

In [1]:
import pandas as pd
import pickle
import re
import numpy as np

## Data

In [2]:
A_train = pd.read_excel("../../data/train_task_C1.xlsx", index_col="id")
A_val = pd.read_excel("../../data/val_task_C1.xlsx", index_col="id")
A_test = pd.read_excel("../../data/test_task_C1.xlsx", index_col="id")

In [3]:
X_train = A_train["Q A".split()]
y_train = A_train["label"]

X_test = A_test["Q A".split()]
y_test = A_test["label"]

X_val = A_val["Q A".split()]
y_val = A_val["label"]

In [4]:
ix = 280
q = X_train["Q"][A_train["label_C1"] == 1].iloc[ix]
a = X_train["A"][A_train["label_C1"] == 1].iloc[ix]
y = y_train[A_train["label_C1"] == 1].iloc[ix]
q, a, y

('En 5 días, las gallinas de una granja pusieron 700 huevos.   ¿Es posible con esta información crear un gráfico de barras o pictograma? Argumenta tu respuesta.',
 'mn\n',
 1)

In [10]:
def clf_rules(q, a):
    if any([x in a.lower() for x in ["caca", ":D", ":c", ":)", ":(", 
                                     "nose", "profe", "tia", "noce", ":v"]]):
        return 1
    else:
        if any([x in q.lower() for x in ["¿cuánta", "¿cuánto","¿cuanta", "¿cuanto", "¿con cuánta", 
                                         "¿con cuanta", "¿con cuánto", "¿con cuanto", "¿a cuántos", 
                                         "¿a cuántas", "¿qué cantidad"]]):
            if any([x in q.lower() for x in ["explica", "explique", "justifique", 
                                             "con tus palabras", "con sus palabras", "en palabras", 
                                             "plantea la", "escribe", "¿qué"]]):
                #Q=2
                nums = len(re.findall(r'[0-9]+', a))
                no_nums = len("".join(a.split()))-nums
                if no_nums>0 and nums>0:
                    return 0
            else:
                #Q=1
                nums = len(re.findall(r'[0-9]+', a))
                if nums>0:
                    return 0
        elif any([x in q.lower() for x in ["¿está correcto", "¿esta correcto", "¿está correcta", 
                                         "¿esta correcta", "¿es correcta", "¿es correcto", 
                                         "correcto?", "correcta?", "equivocado?", "equivocada?",
                                         "¿estará correcto", "¿estara correcto", "¿estara correcta",
                                         "¿estará correcta", "afirmación?", "afirmacion?"]]):
            #Q=3
            aff = len([x for x in ["si", "no", "correcto", "mal", "bien", "por que", "rason", "porque", "razon "] if x in a.lower()])
            if aff>0:
                return 0
        else:
            nums = len(re.findall(r'[0-9]+', a))
            no_nums = len("".join(a.split()))-nums
            num_vow = len([x for x in a if x in "aeiou"])
            if no_nums>0:
                if 0.2<num_vow/no_nums<0.8:
                    return 0
    return 1

def clf_rules_predict(df):
    o = []
    for ix in df.index:
        row = df.loc[ix]
        q, a = row["Q"], row["A"]
        q = str(q)
        a = str(a)
        oi = clf_rules(q, a)
        o.append(oi)
    return np.array(o)

In [6]:
from sklearn.metrics import classification_report

In [11]:
%%time
y_pred = clf_rules_predict(X_train)
report = classification_report(y_train, y_pred, output_dict=True)
train_report = pd.DataFrame(report)
train_report

Wall time: 1.65 s


,0,1,accuracy,macro avg,weighted avg
precision,0.927099,0.480702,0.861061,0.703900,0.867626
recall,0.911368,0.533766,0.861061,0.722567,0.861061
f1-score,0.919166,0.505846,0.861061,0.712506,0.864100
support,10019.000000,1540.000000,0.861061,11559.000000,11559.000000


In [12]:
y_pred = clf_rules_predict(X_val)
report = classification_report(y_val, y_pred, output_dict=True)
val_report = pd.DataFrame(report)
val_report

,0,1,accuracy,macro avg,weighted avg
precision,0.932114,0.504566,0.867495,0.718340,0.874871
recall,0.913546,0.569588,0.867495,0.741567,0.867495
f1-score,0.922736,0.535109,0.867495,0.728923,0.870839
support,2510.000000,388.000000,0.867495,2898.000000,2898.000000


In [13]:
y_pred = clf_rules_predict(X_test)
report = classification_report(y_test, y_pred, output_dict=True)
test_report = pd.DataFrame(report)
test_report

,0,1,accuracy,macro avg,weighted avg
precision,0.881239,0.697917,0.855244,0.789578,0.844412
recall,0.946396,0.492647,0.855244,0.719521,0.855244
f1-score,0.912656,0.577586,0.855244,0.745121,0.845345
support,541.000000,136.000000,0.855244,677.000000,677.000000


In [16]:
y_train.value_counts()

0    10019
1     1540
Name: label, dtype: int64

## Save pickle

In [10]:
# pickle.dump(clf, open("results/clf_rule_based_task_C1.pickle", "wb"))

In [11]:
# pickle.dump(best_params, open("results/best_params_rule_based_task_C1.pickle", "wb"))

In [15]:
pickle.dump(train_report, open("results/train_report_rule_based.pickle", "wb"))

In [16]:
pickle.dump(val_report, open("results/val_report_rule_based.pickle", "wb"))

In [17]:
pickle.dump(test_report, open("results/test_report_rule_based.pickle", "wb"))